In [ ]:
!pip install gtfs2nx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.0 MB/s eta 0:00:00
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.0.1
    Uninstalling geopandas-1.0.1:
      Successfully uninstalled geopandas-1.0.1


In [ ]:
import gtfs2nx as gx
import networkx as nx

G = gx.transit_graph('gtfs_vta.zip')

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
nx.write_gml(G, 'vta.gml')

NetworkXError: <Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
 is not a string

In [ ]:
import gtfs2nx as gx
import networkx as nx

G = gx.transit_graph('gtfs_vta.zip')

# Convert the CRS object to a string representation (e.g., WKT or EPSG code).
# Assuming the CRS object is stored under the key 'crs' in G.graph:
if 'crs' in G.graph:
    # This converts the CRS object to its EPSG code, if available
    G.graph['crs'] = str(G.graph['crs'].to_epsg())
    # If the to_epsg method isn't available you can also try:
    # G.graph['crs'] = G.graph['crs'].to_wkt() # Convert to WKT if available
    # Otherwise you may need to find a more appropriate string
    # representation for your specific CRS object.

nx.write_gml(G, 'vta.gml')

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
import pandas as pd
import networkx as nx

# Load GTFS files
stops = pd.read_csv("stops.txt")
stop_times = pd.read_csv("stop_times.txt")
trips = pd.read_csv("trips.txt")

# Create directed graph
G = nx.nx.MultiDiGraph()

# Add nodes for each stop
for _, row in stops.iterrows():
    G.add_node(row["stop_id"], name=row["stop_name"], lat=row["stop_lat"], lon=row["stop_lon"])

# Add edges for each consecutive stop in the same trip
# Sort by trip_id and stop_sequence to ensure stops are in order
stop_times_sorted = stop_times.sort_values(by=["trip_id", "stop_sequence"])

# Group by trip_id to find consecutive stops within each trip
for trip_id, trip_stops in stop_times_sorted.groupby("trip_id"):
    previous_stop = None
    for _, row in trip_stops.iterrows():
        if previous_stop:
            # Add an edge from previous stop to current stop
            G.add_edge(previous_stop, row["stop_id"], trip_id=trip_id)
        previous_stop = row["stop_id"]

# At this point, G is a directed graph representing the GTFS network.
# You can now analyze or visualize it with NetworkX functions.


nx.write_gml(G, 'vta3.gml')


In [ ]:
import pandas as pd
import networkx as nx

# Load GTFS files
stops = pd.read_csv("stops.txt")
stop_times = pd.read_csv("stop_times.txt")
trips = pd.read_csv("trips.txt")
routes = pd.read_csv("routes.txt")

# Ensure all stop_id columns are strings to avoid duplication from type mismatch
stops["stop_id"] = stops["stop_id"].astype(str)
stop_times["stop_id"] = stop_times["stop_id"].astype(str)
trips["trip_id"] = trips["trip_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Clean the stops data to remove any rows with missing latitude or longitude
stops = stops.dropna(subset=["stop_lat", "stop_lon"])

# Create directed graph
G = nx.MultiDiGraph()


# Add unique nodes for each stop
for _, row in stops.iterrows():
    G.add_node(row["stop_id"], name=row["stop_name"], lat=row["stop_lat"], lon=row["stop_lon"])

# Merge trips and routes to get route names associated with each trip
trips_with_routes = trips.merge(routes, on="route_id")

# Sort stop_times to ensure stops are in order and drop rows with missing stop_id
stop_times_sorted = stop_times.dropna(subset=["stop_id"]).sort_values(by=["trip_id", "stop_sequence"])

# Group by trip_id to find consecutive stops within each trip
for trip_id, trip_stops in stop_times_sorted.groupby("trip_id"):
    trips_with_routes["trip_id"] = trips_with_routes["trip_id"].astype(str)
    # Get the route name for this trip (ensure `route_name` assignment works correctly)
    route_info = trips_with_routes[trips_with_routes["trip_id"] == str(trip_id)]
    route_name = (route_info["route_short_name"].values[0] + " " + route_info["route_long_name"].values[0]) if not route_info.empty else "Unknown Route"

    if route_info["direction_id"].values[0] == 1:
      previous_stop = None
      for _, row in trip_stops.iterrows():
          if previous_stop and row["stop_id"] in G.nodes and previous_stop in G.nodes:
              # Add an edge with route name attribute
              has_edge_with_route_name = any(
                  data.get("route_name") == route_name  # data is defined here
                  for _, _, data in G.edges(nbunch=[previous_stop, row["stop_id"]], data=True)
              )

              if not has_edge_with_route_name:
                G.add_edge(previous_stop, row["stop_id"], trip_id=trip_id, route_name=route_name)
          previous_stop = row["stop_id"]

# Export the graph to Gephi-compatible format
nx.write_gml(G, "transit_network_inbound_multi2.gml")


In [ ]:
# Ensure all trip_id columns are strings to avoid mismatches
stop_times["trip_id"] = stop_times["trip_id"].astype(str)
trips["trip_id"] = trips["trip_id"].astype(str)

# Merge trips and routes to get route names associated with each trip
trips_with_routes = trips.merge(routes, on="route_id")

# Check if trips_with_routes has entries
print("Sample of trips_with_routes:\n", trips_with_routes.head())
print("Unique trip_ids in trips_with_routes:", trips_with_routes["trip_id"].nunique())
print("Unique trip_ids in stop_times:", stop_times["trip_id"].nunique())

# Check if all stop_times trip_ids are in trips_with_routes
missing_trip_ids = set(stop_times["trip_id"]) - set(trips_with_routes["trip_id"])
print("Missing trip_ids:", missing_trip_ids)


Sample of trips_with_routes:
   route_id service_id  trip_id trip_headsign  direction_id  block_id  \
0     Blue    269.0.1  3733714     BAYPOINTE             0    939409   
1     Blue    269.0.1  3733715     BAYPOINTE             0    939410   
2     Blue    269.0.2  3697865     BAYPOINTE             0      9003   
3     Blue    269.0.3  3698087     BAYPOINTE             0      9003   
4     Blue    269.0.1  3733716     BAYPOINTE             0      9003   

   shape_id  wheelchair_accessible  bikes_allowed agency_id route_short_name  \
0    115504                      0              0       VTA        Blue Line   
1    115504                      0              0       VTA        Blue Line   
2    115504                      0              0       VTA        Blue Line   
3    115504                      0              0       VTA        Blue Line   
4    115504                      0              0       VTA        Blue Line   

            route_long_name route_desc  route_type  \
0 

In [ ]:
import pandas as pd
import networkx as nx

# Load GTFS files
stops = pd.read_csv("stops.txt")
stop_times = pd.read_csv("stop_times.txt")
trips = pd.read_csv("trips.txt")
routes = pd.read_csv("routes.txt")

# Ensure all stop_id columns are strings to avoid duplication from type mismatch
stops["stop_id"] = stops["stop_id"].astype(str)
stop_times["stop_id"] = stop_times["stop_id"].astype(str)
trips["trip_id"] = trips["trip_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Clean the stops data to remove any rows with missing latitude or longitude
stops = stops.dropna(subset=["stop_lat", "stop_lon"])

# Create directed graph
G = nx.MultiDiGraph()


# Add unique nodes for each stop
for _, row in stops.iterrows():
    G.add_node(row["stop_id"], name=row["stop_name"], lat=row["stop_lat"], lon=row["stop_lon"])

# Merge trips and routes to get route names associated with each trip
trips_with_routes = trips.merge(routes, on="route_id")

# Sort stop_times to ensure stops are in order and drop rows with missing stop_id
stop_times_sorted = stop_times.dropna(subset=["stop_id"]).sort_values(by=["trip_id", "stop_sequence"])

for trip_id, trip_stops in stop_times_sorted.groupby("trip_id"):
    trips_with_routes["trip_id"] = trips_with_routes["trip_id"].astype(str)
    route_info = trips_with_routes[trips_with_routes["trip_id"] == str(trip_id)]

    if not route_info.empty:
        route_short_name = route_info.get("route_short_name", "Unknown").values[0]
        route_long_name = route_info.get("route_long_name", "Unknown").values[0]
        route_name = f"{route_short_name} {route_long_name}"

        if "direction_id" in route_info.columns and route_info["direction_id"].values[0] == 1:
            previous_stop = None
            for _, row in trip_stops.iterrows():
                if previous_stop and row["stop_id"] in G.nodes and previous_stop in G.nodes:
                    if G.has_edge(previous_stop, row["stop_id"]):
                        edges = G.get_edge_data(previous_stop, row["stop_id"])
                        has_edge_with_route_name = any(data.get("route_name") == route_name for data in edges.values())
                    else:
                        has_edge_with_route_name = False

                    if not has_edge_with_route_name:
                        G.add_edge(previous_stop, row["stop_id"], trip_id=trip_id, route_name=route_name)
                previous_stop = row["stop_id"]


# Export the graph to Gephi-compatible format
nx.write_gml(G, "transit_network_inbound_multi_fix.gml")


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load GTFS files
stops = pd.read_csv("stops.txt")
stop_times = pd.read_csv("stop_times.txt")
trips = pd.read_csv("trips.txt")
routes = pd.read_csv("routes.txt")
calendar = pd.read_csv("calendar.txt")  # Or `calendar_dates.txt` if applicable

# Step 1: Filter services for Monday
monday_services = calendar[calendar["monday"] == 1]["service_id"]

# Step 2: Filter trips for Monday services and one direction
filtered_trips = trips[(trips["service_id"].isin(monday_services)) &
                       (trips["direction_id"] == 1)]

# Step 3: Filter stop_times by trips and time range
# Define the time range (e.g., between 06:00:00 and 10:00:00)
start_time = "00:00:00"
end_time = "23:00:00"

stop_times["arrival_time"] = stop_times["arrival_time"].astype(str)
filtered_stop_times = stop_times[stop_times["trip_id"].isin(filtered_trips["trip_id"])]
filtered_stop_times = filtered_stop_times[
    (filtered_stop_times["arrival_time"] >= start_time) &
    (filtered_stop_times["arrival_time"] <= end_time)
]
filtered_stop_times = filtered_stop_times.sort_values(by=["trip_id", "stop_sequence"])

# Step 4: Create a directed graph
G = nx.DiGraph()  # Use a simple directed graph to ensure unique edges for each path

# Add nodes for stops
for _, row in stops.iterrows():
    G.add_node(row["stop_id"], name=row["stop_name"], lat=row["stop_lat"], lon=row["stop_lon"])

# Group stop_times by route and generate unique paths
trip_route_map = filtered_trips.merge(routes, on="route_id")
route_groups = trip_route_map.groupby("route_id")

for route_id, route_trips in route_groups:
    route_name = f"{route_trips['route_short_name'].iloc[0]} {route_trips['route_long_name'].iloc[0]}"

    # Collect all stops for this route, maintaining stop sequence order
    route_stop_sequences = []
    for trip_id in route_trips["trip_id"]:
        trip_stops = filtered_stop_times[filtered_stop_times["trip_id"] == trip_id]
        stop_sequence = trip_stops.sort_values(by="stop_sequence")["stop_id"].tolist()
        route_stop_sequences.append(stop_sequence)

    # Merge all sequences into a single unique path (remove duplicates)
    unique_path = []
    for sequence in route_stop_sequences:
        for stop in sequence:
            if not unique_path or stop != unique_path[-1]:  # Avoid consecutive duplicates
                unique_path.append(stop)

    # Add edges for the unique path
    for i in range(len(unique_path) - 1):
        G.add_edge(unique_path[i], unique_path[i + 1], route_id=route_id, route_name=route_name)

# Step 5: Export the graph
nx.write_gml(G, "reduced_network_monday.gml")


In [ ]:
!pip install simplekml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=763e95da5880f9918e7073e0ef6bc90f3e75b0be773463efae8b7697f5cdd13e
  Stored in directory: /root/.cache/pip/wheels/16/7c/e6/621a95f24bd1ff987368f1c37881ffe013433ff6b413d87fcb
Successfully built simplekml


In [ ]:
import pandas as pd
import simplekml

# Load GTFS files
stops = pd.read_csv("stops.txt")
routes = pd.read_csv("routes.txt")
trips = pd.read_csv("trips.txt")
stop_times = pd.read_csv("stop_times.txt")
shapes = pd.read_csv("shapes.txt") if "shapes.txt" in locals() else None

# Create a new KML instance
kml = simplekml.Kml()

# Add stops as points
for _, stop in stops.iterrows():
    try:
        stop_id = str(stop["stop_id"])
        stop_name = str(stop["stop_name"])
        lat = float(stop["stop_lat"])
        lon = float(stop["stop_lon"])
        kml.newpoint(name=stop_name, coords=[(lon, lat)])
    except KeyError as e:
        print(f"Skipping stop due to missing key: {e}")
    except ValueError as e:
        print(f"Skipping stop due to invalid value: {e}")

# Add shapes (routes as lines) if `shapes.txt` exists
if shapes is not None:
    grouped_shapes = shapes.groupby("shape_id")
    for shape_id, group in grouped_shapes:
        line = kml.newlinestring(name=f"Shape {shape_id}")
        line.coords = list(zip(group["shape_pt_lon"], group["shape_pt_lat"]))
        line.style.linestyle.color = simplekml.Color.red
        line.style.linestyle.width = 2

# Optionally add trip-route information
trip_route_map = trips.merge(routes, on="route_id")
trip_stop_map = trip_route_map.merge(stop_times, on="trip_id")
for route_id, group in trip_stop_map.groupby("route_id"):
    route_name = group["route_short_name"].iloc[0] + " " + group["route_long_name"].iloc[0]
    description = f"Route: {route_name}, ID: {route_id}"
    folder = kml.newfolder(name=route_name)
    for _, stop in group.iterrows():
        stop_id = str(stop["stop_id"])
        stop_info = stops[stops["stop_id"] == stop_id]
        if not stop_info.empty:
            stop_name = stop_info["stop_name"].iloc[0]
            lat = stop_info["stop_lat"].iloc[0]
            lon = stop_info["stop_lon"].iloc[0]
            folder.newpoint(name=stop_name, coords=[(lon, lat)], description=description)

# Save to KML file
kml.save("gtfs_to_kml.kml")


In [ ]:
!pip install fiona ogr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 36.9 MB/s eta 0:00:00


In [ ]:
!apt-get install -y gdal-bin python3-gdal
!pip install gdal

In [ ]:
import osgeo.ogr as ogr

def kml_to_gml(kml_file, gml_file):
    # Open the KML file
    kml_driver = ogr.GetDriverByName('KML')
    kml_datasource = kml_driver.Open(kml_file)

    # Get the first layer from the KML file
    kml_layer = kml_datasource.GetLayer()

    # Create the GML file
    gml_driver = ogr.GetDriverByName('GML')
    gml_datasource = gml_driver.CreateDataSource(gml_file)

    # Create a new GML layer
    gml_layer = gml_datasource.CreateLayer('layer', geom_type=kml_layer.GetGeomType())

    # Copy features from KML layer to GML layer
    feature = kml_layer.GetNextFeature()
    while feature:
        gml_layer.CreateFeature(feature)
        feature = kml_layer.GetNextFeature()

    # Close the data sources
    kml_datasource = None
    gml_datasource = None

# Example usage
kml_file = 'gtfs_to_kml.kml'
gml_file = 'output_gmrrl_file.gml'
kml_to_gml(kml_file, gml_file)

AttributeError: 'NoneType' object has no attribute 'GetLayer'

In [ ]:
import pandas as pd

def gtfs_to_gephi_gml_with_annotations(stops_file, stop_times_file, routes_file, output_gml_file):
    # Load GTFS data
    stops_df = pd.read_csv(stops_file)
    stop_times_df = pd.read_csv(stop_times_file)
    routes_df = pd.read_csv(routes_file)

    # Start writing the GML file
    with open(output_gml_file, 'w') as f:
        f.write("graph [\n")

        # Create nodes from stops
        node_id_map = {}  # Mapping from stop_id to node_id
        node_id = 1
        for _, row in stops_df.iterrows():
            stop_id = row['stop_id']
            stop_name = row['stop_name']
            stop_lat = row['stop_lat']
            stop_lon = row['stop_lon']

            # Write node with attributes for stop name and location
            f.write(f'  node [\n    id {node_id}\n    label "{stop_name}"\n')
            f.write(f'    x {stop_lon}\n    y {stop_lat}\n')
            f.write(f'    stop_id "{stop_id}"\n    stop_name "{stop_name}"\n')
            f.write(f'  ]\n')

            # Map stop_id to node_id
            node_id_map[stop_id] = node_id
            node_id += 1

        # Create edges from stop_times (edges represent trips between stops)
        for _, row in stop_times_df.iterrows():
            trip_id = row['trip_id']
            stop_id = row['stop_id']
            arrival_time = row['arrival_time']
            departure_time = row['departure_time']

            # Get route_id from the trip (assume trip_id exists in the 'routes.txt' file)
            route_id = None
            route_name = None
            route_row = routes_df[routes_df['route_id'] == trip_id]
            if not route_row.empty:
                route_id = route_row.iloc[0]['route_id']
                route_name = route_row.iloc[0]['route_long_name']

            # Get node ids for source and target
            source_node_id = node_id_map.get(stop_id)
            target_node_id = None  # Find the next stop in the trip

            # You may need to get the next stop in the trip for creating edges
            next_stop = stop_times_df[(stop_times_df['trip_id'] == trip_id) & (stop_times_df['stop_sequence'] == row['stop_sequence'] + 1)]
            if not next_stop.empty:
                next_stop_id = next_stop.iloc[0]['stop_id']
                target_node_id = node_id_map.get(next_stop_id)

            if source_node_id and target_node_id:
                # Write edge with annotations: trip_id, route_id, route_name, arrival/departure times
                f.write(f'  edge [\n    source {source_node_id}\n    target {target_node_id}\n')
                f.write(f'    label "Trip {trip_id} - Route {route_id} ({route_name})"\n')
                f.write(f'    trip_id "{trip_id}"\n    route_id "{route_id}"\n')
                f.write(f'    route_name "{route_name}"\n')
                f.write(f'    arrival_time "{arrival_time}"\n    departure_time "{departure_time}"\n')
                f.write(f'  ]\n')

        f.write("]\n")

# Example usage
gtfs_to_gephi_gml_with_annotations('stops.txt', 'stop_times.txt', 'routes.txt', 'output_gephi_graph_with_annotations.gml')


In [ ]:
!pip install peartree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
import peartree as pt
path = 'gtfs_vta.zip'

# Automatically identify the busiest day and
# read that in as a Partidge feed
feed = pt.get_representative_feed(path)

# Set a target time period to
# use to summarize impedance
start = 0*60*60  # 7:00 AM
end = 12*60*60  # 10:00 AM

# Converts feed subset into a directed
# network multigraph
G = pt.load_feed_as_graph(feed, start, end)

#transform G into gml
nx.write_gml(G, "gtfs_vta.gml")

AttributeError: 'Series' object has no attribute 'iteritems'

In [ ]:
!pip install peartree --upgrade
import peartree as pt
import networkx as nx # Import networkx

path = 'gtfs_vta.zip'

# Automatically identify the busiest day and
# read that in as a Partidge feed
feed = pt.get_representative_feed(path)

# Set a target time period to
# use to summarize impedance
start = 0*60*60  # 7:00 AM
end = 12*60*60  # 10:00 AM

# Converts feed subset into a directed
# network multigraph
G = pt.load_feed_as_graph(feed, start, end)

#transform G into gml
nx.write_gml(G, "gtfs_vta.gml") # Use nx.write_gml to write to GML format

AttributeError: 'Series' object has no attribute 'iteritems'

In [ ]:
!pip install pandas==1.5.3  # Downgrade to a pandas version with iteritems
!pip install peartree
import peartree as pt
import networkx as nx

path = 'gtfs_vta.zip'

feed = pt.get_representative_feed(path)

start = 0*60*60
end = 12*60*60

G = pt.load_feed_as_graph(feed, start, end)

fig, ax = pt.generate_plot(G)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: ite

UnboundLocalError: local variable 'ox' referenced before assignment

In [ ]:
!pip install osmnx

In [ ]:
import peartree as pt
import networkx as nx
import osmnx as ox # Import osmnx library

path = 'gtfs_vta.zip'

feed = pt.get_representative_feed(path)

start = 0*60*60
end = 12*60*60

G = pt.load_feed_as_graph(feed, start, end)

nx.write_gml(G, "gtfs_vta_network.gml") # Use nx.write_gml to write to GML format

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: ite

NetworkXError: CRS.from_epsg(4326) is not a string

In [ ]:
!pip install osmnx
import peartree as pt
import networkx as nx
import osmnx as ox

path = 'gtfs_vta.zip'

feed = pt.get_representative_feed(path)

start = 0*60*60
end = 12*60*60

G = pt.load_feed_as_graph(feed, start, end)

# Instead of plotting, write the graph to a GML file
nx.write_gml(G, "gtfs_vta_network.gml")
print("Graph written to gtfs_vta_network.gml")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: ite

NetworkXError: CRS.from_epsg(4326) is not a string

In [ ]:
import peartree as pt
import networkx as nx
import osmnx as ox

path = 'gtfs_vta.zip'

feed = pt.get_representative_feed(path)

start = 0*60*60
end = 12*60*60

G = pt.load_feed_as_graph(feed, start, end)

# Before writing, remove or convert non-string attributes from G.graph
for key in list(G.graph.keys()): #Iterate over a copy of the keys to allow deletion
    if not isinstance(G.graph[key], str):
        #If the value is not a string, remove it
        del G.graph[key]
        print(f"Removed non-string attribute '{key}' from graph metadata.")

# Instead of plotting, write the graph to a GML file
nx.write_gml(G, "gtfs_vta_network.gml")
print("Graph written to gtfs_vta_network.gml")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: ite

Removed non-string attribute 'crs' from graph metadata.
Graph written to gtfs_vta_network.gml


/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for node_id, dist_val in nearest_nodes.iteritems():
/usr/local/lib/python3.10/dist-packages/peartree/graph.py:197: FutureWarning: iteritems is deprecated and will be removed in a futur

In [ ]:
!pip install cartopy.crs

ERROR: Could not find a version that satisfies the requirement cartopy.crs (from versions: none)
ERROR: No matching distribution found for cartopy.crs


In [ ]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 42.7 MB/s eta 0:00:00


In [ ]:
"""
Created on Tue May  8 17:48:55 2018

@author: roland

inspired by https://github.com/paulgb/gtfs-gexf/blob/master/transform.py
"""

import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from csv import DictReader
from itertools import groupby
import cartopy.feature as cfeature


TRIPS_FILE = 'trips.txt'
ROUTES_FILE = 'routes.txt'
STOPS_FILE = 'stops.txt'

INCLUDE_AGENCIES=['VTA']

IGNORE_ROUTE=[]

G = nx.MultiGraph()

def get_stop_id(stop_id):
    """ translate stop_id to parent_stop_id
        if available
    """
    if STOPS[stop_id]['parent_station'] == '':
        return stop_id
    else:
        return STOPS[stop_id]['parent_station']


def add_stop_to_graph(G, stop_id):
    """ add stop as new node to graph
    """
    #lookup details of the stop (parent stop if available)
    node = STOPS[get_stop_id(stop_id)]

    if node['stop_id'] not in G.nodes:
        G.add_node(node['stop_id'],
                   stop_name = node['stop_name'],
                   lon = float(node['stop_lon']),
                   lat = float(node['stop_lat']))
    return G

def add_edge_to_graph(G, from_id, to_id, route_short_name):
    """ add edge to graph
        adding the route short name as a key
        if the edge and key exist, increment the count
    """
    edge = G.get_edge_data(get_stop_id(from_id), get_stop_id(to_id),route_short_name, default = 0)
    if edge == 0:
        G.add_edge(get_stop_id(from_id), get_stop_id(to_id),
                   key=route_short_name,
                   count=1)
    else:
        G.add_edge(get_stop_id(from_id), get_stop_id(to_id),
                   key=route_short_name,
                   count=edge['count']+1)


def load_routes(filename):
    """ include only routes from agencies we are interested in
    """
    routes_csv = DictReader(open(filename,'r'))
    routes_dict = dict()
    for route in routes_csv:
        if (route['agency_id'] in INCLUDE_AGENCIES and
            route['route_id'] not in IGNORE_ROUTE ):

            routes_dict[route['route_id']] = route
    print ('routes', len(routes_dict))
    return routes_dict


def load_trips(filename, routes_dict):
    """ load trips from file
        only include trips on routes we are interested in
    """
    trips_csv = DictReader(open(filename,'r'))
    trips_dict = dict()
    for trip in trips_csv:
        if trip['route_id'] in routes_dict:
            trip['color'] = routes_dict[trip['route_id']]['route_color']
            trip['route_short_name']=routes_dict[trip['route_id']]['route_short_name']
            trips_dict[trip['trip_id']] = trip
    print ('trips', len(trips_dict))
    return trips_dict


def load_stops(filename):
    stops_csv = DictReader(open(filename,'r'))
    stops_dict = dict()
    for stop in stops_csv:
        stops_dict[stop['stop_id']] = stop
    print ('stops', len(stops_dict))
    return stops_dict


# ==============================================


ROUTES = load_routes(filename=ROUTES_FILE)
TRIPS = load_trips(filename=TRIPS_FILE, routes_dict=ROUTES)
STOPS = load_stops(filename=STOPS_FILE)

stop_times_csv = DictReader(open('stop_times.txt','r'))

stops = set()
edges = dict()
for trip_id, stop_time_iter in groupby(stop_times_csv, lambda stop_time: stop_time['trip_id']):
    if trip_id in TRIPS:
        trip = TRIPS[trip_id]
        prev_stop = next(stop_time_iter)['stop_id']
        stops.add(prev_stop)
        for stop_time in stop_time_iter:
            stop = stop_time['stop_id']
            edge = (prev_stop, stop)
            edges[edge] = trip['route_short_name']
            stops.add(stop)
            prev_stop = stop
print ('stops', len(stops))
print ('edges', len(edges))

for stop_id in STOPS:
    if stop_id in stops:
       add_stop_to_graph(G, stop_id)
print('Nodes:', G.number_of_nodes() )

for (start_stop_id, end_stop_id), route_short_name in edges.items():
    add_edge_to_graph(G,
                      from_id = start_stop_id,
                      to_id = end_stop_id,
                      route_short_name=route_short_name)
print('Edges:', G.number_of_edges() )








deg = nx.degree(G)
labels = {stop_id: G.nodes[stop_id]['stop_name'] if deg[stop_id] >= 0 else ''
          for stop_id in G.nodes}

pos = {stop_id: (G.nodes[stop_id]['lon'], G.nodes[stop_id]['lat'])
       for stop_id in G.nodes}

nx.write_gml(G, "gtfs_vta_network_new.gml")
print("Graph written to gtfs_vta_network.gml")

plt.show()

routes 76
trips 8612
stops 3355
stops 3277
edges 3706
Nodes: 3158
Edges: 3656
Graph written to gtfs_vta_network.gml


In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from csv import DictReader
from itertools import groupby

# File paths
TRIPS_FILE = 'trips.txt'
ROUTES_FILE = 'routes.txt'
STOPS_FILE = 'stops.txt'
CALENDAR_FILE = 'calendar.txt'
CALENDAR_DATES_FILE = 'calendar_dates.txt'
STOP_TIMES_FILE = 'stop_times.txt'

INCLUDE_AGENCIES = ['VTA']
IGNORE_ROUTE = []

G = nx.MultiGraph()


# Helper functions
def get_stop_id(stop_id):
    """Translate stop_id to parent_stop_id if available."""
    if STOPS[stop_id]['parent_station'] == '':
        return stop_id
    else:
        return STOPS[stop_id]['parent_station']


def add_stop_to_graph(G, stop_id):
    """Add stop as new node to graph."""
    node = STOPS[get_stop_id(stop_id)]
    if node['stop_id'] not in G.nodes:
        G.add_node(node['stop_id'],
                   stop_name=node['stop_name'],
                   lon=float(node['stop_lon']),
                   lat=float(node['stop_lat']))
    return G


def add_edge_to_graph(G, from_id, to_id, route_short_name):
    """Add edge to graph, incrementing the count if it exists."""
    edge = G.get_edge_data(get_stop_id(from_id), get_stop_id(to_id), route_short_name, default=0)
    if edge == 0:
        G.add_edge(get_stop_id(from_id), get_stop_id(to_id),
                   key=route_short_name,
                   count=1)
    else:
        G.add_edge(get_stop_id(from_id), get_stop_id(to_id),
                   key=route_short_name,
                   count=edge['count'] + 1)


# Data loading functions
def load_routes(filename):
    routes_csv = DictReader(open(filename, 'r'))
    routes_dict = {}
    for route in routes_csv:
        if (route['agency_id'] in INCLUDE_AGENCIES and
                route['route_id'] not in IGNORE_ROUTE):
            routes_dict[route['route_id']] = route
    print('routes', len(routes_dict))
    return routes_dict


def load_trips(filename, routes_dict):
    trips_csv = DictReader(open(filename, 'r'))
    trips_dict = {}
    for trip in trips_csv:
        if trip['route_id'] in routes_dict:
            trip['color'] = routes_dict[trip['route_id']]['route_color']
            trip['route_short_name'] = routes_dict[trip['route_id']]['route_short_name']
            trips_dict[trip['trip_id']] = trip
    print('trips', len(trips_dict))
    return trips_dict


def load_stops(filename):
    stops_csv = DictReader(open(filename, 'r'))
    stops_dict = {}
    for stop in stops_csv:
        stops_dict[stop['stop_id']] = stop
    print('stops', len(stops_dict))
    return stops_dict


def load_calendar(filename):
    calendar_csv = DictReader(open(filename, 'r'))
    calendar_dict = {}
    for calendar in calendar_csv:
        calendar_dict[calendar['service_id']] = calendar
    print('calendar', len(calendar_dict))
    return calendar_dict


def load_calendar_dates(filename):
    calendar_dates_csv = DictReader(open(filename, 'r'))
    calendar_dates_dict = {}
    for date in calendar_dates_csv:
        if date['exception_type'] == '1':  # 1 means added service
            calendar_dates_dict[date['service_id']] = date
    print('calendar_dates', len(calendar_dates_dict))
    return calendar_dates_dict


def operates_on_monday(service_id):
    """Check if a trip operates on Monday."""
    if service_id in CALENDAR:
        return CALENDAR[service_id]['monday'] == '1'
    return False


def filter_trips_for_monday_and_direction(trips_dict, direction):
    """Filter trips for Monday and a specific direction."""
    monday_trips = {}
    for trip_id, trip in trips_dict.items():
        if operates_on_monday(trip['service_id']) and int(trip['direction_id']) == direction:
            monday_trips[trip_id] = trip
    return monday_trips


# Load data
ROUTES = load_routes(ROUTES_FILE)
TRIPS = load_trips(TRIPS_FILE, ROUTES)
STOPS = load_stops(STOPS_FILE)
CALENDAR = load_calendar(CALENDAR_FILE)
CALENDAR_DATES = load_calendar_dates(CALENDAR_DATES_FILE)

# Filter for Monday trips in a specific direction (e.g., inbound)
DIRECTION = 0  # 0 for inbound, 1 for outbound
MONDAY_TRIPS = filter_trips_for_monday_and_direction(TRIPS, DIRECTION)

# Process stop times and build the graph
stop_times_csv = DictReader(open(STOP_TIMES_FILE, 'r'))
stops = set()
edges = {}
for trip_id, stop_time_iter in groupby(stop_times_csv, lambda stop_time: stop_time['trip_id']):
    if trip_id in MONDAY_TRIPS:  # Only process trips filtered for Monday and direction
        trip = MONDAY_TRIPS[trip_id]
        prev_stop = next(stop_time_iter)['stop_id']
        stops.add(prev_stop)
        for stop_time in stop_time_iter:
            stop = stop_time['stop_id']
            edge = (prev_stop, stop)
            edges[edge] = trip['route_short_name']
            stops.add(stop)
            prev_stop = stop

print('stops', len(stops))
print('edges', len(edges))

for stop_id in STOPS:
    if stop_id in stops:
        add_stop_to_graph(G, stop_id)
print('Nodes:', G.number_of_nodes())

for (start_stop_id, end_stop_id), route_short_name in edges.items():
    add_edge_to_graph(G,
                      from_id=start_stop_id,
                      to_id=end_stop_id,
                      route_short_name=route_short_name)
print('Edges:', G.number_of_edges())

# Write graph to file
nx.write_gml(G, "gtfs_vta_network_monday_direction.gml")
print("Graph written to gtfs_vta_network_monday_direction.gml")


routes 76
trips 8612
stops 3355
calendar 92
calendar_dates 44
stops 1869
edges 1989
Nodes: 1811
Edges: 1989
Graph written to gtfs_vta_network_monday_direction.gml


In [2]:
#remove random nodes from the graph and output it as gml
G_copy = G.copy()
import random
import networkx as nx
import matplotlib.pyplot as plt


random.seed(42)  # Set a seed for reproducibility

# Set the percentage of nodes to remove (e.g., 10%)
percentage_to_remove = 0.1
num_nodes_to_remove = int(len(G_copy.nodes) * percentage_to_remove)

# Randomly select nodes to remove
nodes_to_remove = random.sample(list(G_copy.nodes), num_nodes_to_remove)

# Print nodes being removed
print(f"Nodes to remove: {nodes_to_remove}")

# Remove selected nodes from the graph
G_copy.remove_nodes_from(nodes_to_remove)

# Save or analyze the modified graph
nx.write_gml(G_copy, "graph_after_random_removal.gml")
print(f"Graph now has {G_copy.number_of_nodes()} nodes and {G_copy.number_of_edges()} edges.")

Nodes to remove: ['3794', '662', '159', '4576', '1577', '1301', '1130', '737', '4550', '597', '4167', '4574', '3348', '504', '3527', '2527', '224', '217', '572', '1113', '1253', '3001', '3609', '210', '3387', '1037', '4431', '3907', '4347', '2522', '1117', '2730', '3524', '1583', '5541', '5969', '43', 'PS_JAPN', '5514', '899', '4340', '2528', '2070', '844', '1106', 'PS_TASM', '2033', '570', '2339', '580', '2209', 'PS_ERTC', '2080', '3613', '1467', '5517', '330', '4469', '2776', '3226', '698', '2336', '412', '3366', '1707', '5656', '3722', '3679', '5932', '2233', '3495', '1021', '4353', '387', '335', '3961', '1229', '4877', '1692', '414', '5903', '5954', '593', '2741', '3784', '5708', '2241', '907', '2283', '2201', '1093', '4023', '1499', '4348', '4187', '3903', '394', '3626', '3782', '925', '3218', '4468', '908', '2782', '2338', '1549', '3795', '4198', '3376', '1115', '4190', '1958', '5760', 'PS_VIEN', '4994', '357', '1246', '5631', '5507', '1892', '2450', '1515', '378', '1096', '3400'

In [3]:
#remove random nodes from the graph and output it as gml
G_copy = G.copy()
import random
import networkx as nx
import matplotlib.pyplot as plt


random.seed(42)  # Set a seed for reproducibility

# Set the percentage of nodes to remove (e.g., 10%)
percentage_to_remove = 0.1


# Make a list of the ten nodes with highest degree
top_ten_nodes = sorted(G_copy.degree, key=lambda x: x[1], reverse=True)[:10]


# Print nodes being removed
print(f"Nodes to remove: {top_ten_nodes}")

# Remove selected nodes from the graph
G_copy.remove_nodes_from(top_ten_nodes)

# Save or analyze the modified graph
nx.write_gml(G_copy, "graph_after_targeted_removal.gml")
print(f"Graph now has {G_copy.number_of_nodes()} nodes and {G_copy.number_of_edges()} edges.")

Nodes to remove: [('PS_MBRT', 17), ('PS_SCTC', 10), ('PS_ERTC', 10), ('PS_DRDN', 9), ('4692', 8), ('413', 7), ('PS_BERR', 7), ('PS_GILR', 6), ('PS_PATC', 6), ('335', 6)]
Graph now has 1811 nodes and 1989 edges.
